# 7.4.2  增强Agent检索能力

In [21]:
%pip -q install  openai langchain langchain_community langchain_openai

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3 -> 23.3.2
[notice] To update, run: C:\Users\freestone\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [22]:

from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI




In [23]:
import os
os.environ["OPENAI_API_KEY"] = "填入你的OpenAI密钥"
os.environ["SERPAPI_API_KEY"] = "填入你的谷歌搜索密钥"


llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613")


加载文档

In [24]:
doc_path = "./state_of_the_union.txt"

from langchain_community.document_loaders import TextLoader

loader = TextLoader(doc_path,encoding="utf-8")
documents = loader.load()


In [25]:
documents

[Document(page_content='Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  \n\nLast year COVID-19 kept us apart. This year we are finally together again. \n\nTonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. \n\nWith a duty to one another to the American people to the Constitution. \n\nAnd with an unwavering resolve that freedom will always triumph over tyranny. \n\nSix days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated. \n\nHe thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined. \n\nHe met the Ukrainian people. \n\nFrom President Zelenskyy to every Ukrainian, their fearlessness, their courage, their determination, inspires the world. \n\nGroups of citize

### 切割分块

In [26]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)


In [ ]:
texts

存储为向量格式

In [28]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

In [ ]:
%pip install chromadb

In [31]:

docsearch = Chroma.from_documents(texts, embeddings, 
collection_name="state-of-union")


实例化RetrievalQA链组件，运行链。

In [32]:
state_of_union = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=docsearch.as_retriever()
)


接着创建另外一个链，从在线网站上加载文档。

In [33]:
from langchain.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://beta.ruff.rs/docs/faq/")

docs = loader.load()
ruff_texts = text_splitter.split_documents(docs)
ruff_db = Chroma.from_documents(ruff_texts, embeddings, 
collection_name="ruff")
ruff = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=ruff_db.as_retriever()
)



Created a chunk of size 2134, which is longer than the specified 1000
Created a chunk of size 1915, which is longer than the specified 1000
Created a chunk of size 1259, which is longer than the specified 1000
Created a chunk of size 1543, which is longer than the specified 1000


正式开始创建Agent组件导入创建 Agent需要的所有类

In [35]:
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.tools import BaseTool
from langchain.chat_models import ChatOpenAI
from langchain import LLMMathChain, SerpAPIWrapper


定义可用的工具。

In [36]:
tools = [
    Tool(
        name="State of Union QA System",
        func=state_of_union.run,
        description="useful for when you need to answer questions about the most recent state of the union address. Input should be a fully formed question.",
    ),
    Tool(
        name="Ruff QA System",
        func=ruff.run,
        description="useful for when you need to answer questions about ruff(a python linter). Input should be a fully formed question.",
    ),
]

    


初始化一个 Agent组件。

In [37]:
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)


C:\Users\freestone\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


开始提问。测试 State of Union QA System 文档的检索。

In [39]:
agent.run(
"What did biden say about ketanji brown jackson in the state of the union address?"
)


C:\Users\freestone\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
I need to find information about what Biden said about Ketanji Brown Jackson in the State of the Union address.
Action: State of Union QA System
Action Input: "What did Biden say about Ketanji Brown Jackson in the State of the Union address?"
Observation: In the State of the Union address, President Biden mentioned that he nominated Circuit Court of Appeals Judge Ketanji Brown Jackson to serve on the United States Supreme Court. He described her as one of the nation's top legal minds who will continue Justice Breyer's legacy of excellence.
Thought:I now know the final answer.
Final Answer: In the State of the Union address, President Biden mentioned that he nominated Circuit Court of Appeals Judge Ketanji Brown Jackson to serve on the United States Supreme Court. He described her as one of the nation's top legal minds who will continue Justice Breyer's legacy of excellence.

> Finished chain.


"In the State of the Union address, President Biden mentioned that he nominated Circuit Court of Appeals Judge Ketanji Brown Jackson to serve on the United States Supreme Court. He described her as one of the nation's top legal minds who will continue Justice Breyer's legacy of excellence."

再次提问。测试 ruff 文档的检索。

In [40]:
agent.run("Why use ruff over flake8?")   



> Entering new AgentExecutor chain...
I need to compare the benefits of using ruff over flake8.
Action: Ruff QA System
Action Input: "What are the benefits of using ruff over flake8?"
Observation: There are several benefits of using Ruff over Flake8:

1. More rules: Ruff implements over 700 rules, compared to Flake8's ~409 rules. This means that Ruff can catch a wider range of code issues and enforce more coding standards.

2. Automatic fixing: Ruff is capable of automatically fixing its own lint violations, which can save you time and effort in manually fixing issues.

3. Native implementation: Ruff implements all rules natively, whereas Flake8 relies on plugins. This means that you don't need to install and manage multiple plugins with Ruff.

4. Compatibility with other tools: Ruff is designed to work well with other tools like Black, Mypy, Pyright, and Pyre. It can be used alongside these tools to provide faster feedback on lint violations while the other tools provide more detail

'The benefits of using Ruff over Flake8 include more rules, automatic fixing capabilities, native implementation, compatibility with other tools, support for multiple Python versions, and easy installation.'